**Answer with the type of stats test you would use (assume normal distribution)**

A. Is there a relationship between the length of your arm and the length of your foot?

B. Do guys and gals quit their jobs at the same rate?

C. Does the length of time of the lecture correlate with a students grade?

**Use the telco_churn data.**

A. Does tenure correlate with monthly charges?

B. Total charges?

C. What happens if you control for phone and internet service?

**Use the employees database.**

A. Is there a relationship between how long an employee has been with the company and their salary?

B. Is there a relationship between how long an employee has been with the company and the number of titles they have had?

**Use the sleepstudy data.**

A. Is there a relationship between days and reaction time?